### Author: Kubam Ivo 
### Purpose: Algorithms For Big Data Project
### Date: 25/3/2021

# Eliminate points

**Algorithm 1(eliminate points-m):** <br>
    **Input:** p1,p2,... , pn' (in order) where n' is the number of points in the stream.<br> 
    **Output**: Skyline points S' <br>
    1. Let x = 24m. 
    2. **Pass 1:** For j : 1, 2, ..., x, let p'j be a point picked uniformly at random from the stream. <br>
    Let S be the set of such points.<br>
    **Pass 2**
    4. for i = 1, ..., n' do 
         * for any p'j, if pi dominates p'j then p'j:=pi
    6. end for 
    7. Let S'={p'1,p'2,...,p'x}.
    8. **Pass 3** 
            Delete from stream all points in S' and all points dominated by any point in S'.
    9. return S' 

In [1]:
# generate points
import random

def generate_points(n):
    data = [(random.randint(1,100),random.randint(1,100)) for x in range(n)] 
    print('Stream contains', len(data), 'data points')
    return data

#stream = generate_points(1000)



In [2]:
# Class for algorithm 1: Eliminate-points (m)
import random
import numpy as np
class Eliminate:
    
    """ Class to generate m skyline points from n stream data """
    def __init__(self, m=3):
        self._m = m
        self._x = self._m * 24

    
    #reservoir sampling

    def reservoir_sample(self, stream):
        """Receives the sample generated data points and does to a reservoir sampling to return selected points """
        k = int(24*self._m)
        if k <  len(stream):
            reservoir = [stream[i] for i in range(k)]
        else:
            reservoir = stream[:]

        for i in range(k,len(stream)):
            j = random.randint(1,i)
            if j < k:
                reservoir[j] = stream[i]
        print(len(reservoir), 'points have been sample from the main stream into the reservoir sample')
        return reservoir

    # dominant points

    def dominate(self, stream, reservoir_point):
        """ Reeceives the selected points from reservoir sampling and replace any if dominated by a point in the stream data """
        dominant_point = reservoir_point [:]
        n = 0
        for i in range(len(stream)):
            sampled_elem = random.choice(dominant_point)

            x1, y1 = sampled_elem
            x2, y2 = stream[i]

            if (x2 >= x1 and y2 >= y1) and (x2 > x1 or y2 > y1):
                n += 1
                #print('Point', stream[i], 'dominates and replaces', sampled_elem)        
                dominant_point[dominant_point.index(sampled_elem)] = stream[i]
        print('There were ', n, ' replacement done between the main stream and reservoir sample')
        print('Preview of skyline points ')
        print(dominant_point[:10], '...')
        return dominant_point

    # Final pass
    def remove_point_stream(self, stream, skyline_points):
        """Delete from stream data all points dominated by points skyline points or points found in skyline points"""
        output_stream = []
        n = 0
        for point in stream:
            if point not in skyline_points:
                n +=1
                output_stream.append(point)

        n = 0
        for point in skyline_points:
            x2, y2 = point
        
            for elem in output_stream:
                x1, y1 = elem
                if (x2 >= x1 and y2 >= y1) and (x2 > x1 or y2 > y1):
                    output_stream.remove(elem)
                    n +=1
        print(n, 'points in the main stream were remove eihter because they were dominated or found in the skyline list')
        print(len(output_stream), ' points are left in the stream')
        return output_stream
    
    


In [3]:

import math

for size in range(1000,10000,1000):
    stream = generate_points(size)
    
#n= len(output_stream)
    m = 2
    test = Eliminate(m)
    reservoir_pts = test.reservoir_sample(stream)
    sky_pts = test.dominate(stream,reservoir_pts)
    output_stream = test.remove_point_stream(stream,sky_pts)
    print(output_stream)
    print()

Stream contains 1000 data points
48 points have been sample from the main stream into the reservoir sample
There were  84  replacement done between the main stream and reservoir sample
Preview of skyline points 
[(99, 84), (64, 100), (91, 89), (79, 23), (61, 75), (84, 77), (75, 73), (60, 82), (47, 93), (100, 63)] ...
949 points in the main stream were remove eihter because they were dominated or found in the skyline list
4  points are left in the stream
[(100, 38), (100, 16), (100, 67), (100, 40)]

Stream contains 2000 data points
48 points have been sample from the main stream into the reservoir sample
There were  81  replacement done between the main stream and reservoir sample
Preview of skyline points 
[(99, 94), (91, 97), (91, 57), (97, 49), (21, 88), (94, 98), (11, 100), (57, 100), (68, 97), (96, 76)] ...
1942 points in the main stream were remove eihter because they were dominated or found in the skyline list
4  points are left in the stream
[(100, 32), (100, 60), (94, 99), (95,

### Lemma Two is proven as the number of points left in the stream is always at most the input stream size divided by 4 or atleast 3n/4 points are eliminated from the main stream

# Streaming RAND

Algorithm 2 (Streaming RAND): 
    1: Let n be the number of points in the input stream. 
    Let m' = 1. 
    2: while the input stream is not empty do: 
    3: let n' be the current number of points in the stream 
    4: Call eliminate points (m'log(nlogn))
    5: If more than n'/2 points are left in the stream, m' = 2 m'
    6: end while 
    Remark: In case the stream cannot be changed, we do not have to actually delete points from stream. 
    We only keep the skyline points found so far and consider only points in the stream that is not dominated by any found skyline points. 
        

In [6]:
import math
m_prime = 1

for size in range(1000,10000,1000):
    stream = generate_points(size)
    n = len(stream)

    while n > 0:
        parameter = int(m_prime*math.log(n*math.log(n)))
        test = Eliminate(parameter)
        reservoir_pts = test.reservoir_sample(stream)
        sky_pts = test.dominate(stream,reservoir_pts)
        output_stream = test.remove_point_stream(stream,sky_pts)
        n_prime = len(output_stream)
        if n_prime > n/2:
            m_prime = 2*m_prime
            n = n_prime
        else:
            print(output_stream)
            print('Current value of m-prime is ',m_prime,"\n")
            break


    


Stream contains 1000 data points
192 points have been sample from the main stream into the reservoir sample
There were  144  replacement done between the main stream and reservoir sample
Preview of skyline points 
[(81, 93), (81, 87), (56, 51), (93, 33), (89, 85), (45, 55), (85, 87), (94, 98), (64, 95), (96, 54)] ...
817 points in the main stream were remove eihter because they were dominated or found in the skyline list
0  points are left in the stream
[]
Current value of m-prime is  1 

Stream contains 2000 data points
216 points have been sample from the main stream into the reservoir sample
There were  217  replacement done between the main stream and reservoir sample
Preview of skyline points 
[(95, 98), (88, 87), (95, 40), (100, 44), (87, 41), (74, 91), (96, 99), (55, 100), (100, 87), (51, 78)] ...
1776 points in the main stream were remove eihter because they were dominated or found in the skyline list
0  points are left in the stream
[]
Current value of m-prime is  1 

Stream c

### Proofs lemma 6: E.g with n = 10000, The probability that the algorithm repeats until m-prime ≥ 2m is at most 1/10000 = . This implies that, with probability (1 − 1/10000)=0.999, the stream will be empty before m-prime is increased again. 

# Fixed Window

In [8]:
# varying the stream size
import time
import math

for size in range(10000,100000,10000):
    stream = generate_points(size)
    w = int(0.01 * len(stream)) # getting the number of points using window size of 0.01.
    output_stream = stream[:] # Assuming output and stream data at thesame at the beginning
    n = len(output_stream)

    tic = time.perf_counter()  # starts the timer
    # simple running the elimate algorithm with w/24
    while n > 0:
        test = Eliminate(w/24)
        reservoir_pts = test.reservoir_sample(output_stream)
        sky_pts = test.dominate(output_stream,reservoir_pts)
        output_stream = test.remove_point_stream(output_stream,sky_pts)
        n = len(output_stream)
    toc = time.perf_counter()
    print("Execution time: ", toc - tic)
    print("\n")
    

Stream contains 10000 data points
100 points have been sample from the main stream into the reservoir sample
There were  213  replacement done between the main stream and reservoir sample
Preview of skyline points 
[(100, 93), (87, 100), (95, 61), (97, 96), (99, 98), (96, 100), (99, 92), (99, 98), (92, 96), (56, 100)] ...
9845 points in the main stream were remove eihter because they were dominated or found in the skyline list
0  points are left in the stream
Execution time:  0.49305660000004536


Stream contains 20000 data points
200 points have been sample from the main stream into the reservoir sample
There were  439  replacement done between the main stream and reservoir sample
Preview of skyline points 
[(99, 88), (95, 99), (95, 72), (63, 100), (100, 98), (97, 74), (88, 89), (94, 92), (92, 99), (87, 92)] ...
19575 points in the main stream were remove eihter because they were dominated or found in the skyline list
0  points are left in the stream
Execution time:  2.181096500000023